In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


# MMAction2 Tutorial

Welcome to MMAction2! This is the official colab tutorial for using MMAction2. In this tutorial, you will learn
- Perform inference with a MMAction2 recognizer.
- Train a new recognizer with a new dataset.


Let's start!

## Install MMAction2

In [2]:
# # Check nvcc version
# !nvcc -V
# # Check GCC version
# !gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
# # install dependencies: (if your colab has CUDA 11.8)
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.2/819.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 2.

In [1]:
# # install MMEngine, MMCV and MMDetection using MIM
# %pip install -U openmim
# !mim install mmengine
# !mim install "mmcv>=2.0.0"

# # Install mmaction2
# !rm -rf mmaction2
# !git clone https://github.com/open-mmlab/mmaction2.git -b main
# %cd mmaction2

# !pip install -e .

# # Install some optional requirements
# !pip install -r requirements/optional.txt

Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.2.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.2.0/index.html
  Using cached mmcv-2.1.0.tar.gz (471 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for mmcv
  Running setup.py clean for mmcv
Failed to build mmcv
ERROR: Could not build wheels for mmcv, which is required to install pyproject.toml-based projects
Cloning into 'mmaction2'...
remote: Enumerating objects: 22864, done.
remote: Total 22864 (delta 0), reused 0 (delta 0), pack-reused 22864
Receiving objects: 100% (22864/22864), 69.62 MiB | 32.94 MiB/s, done.
Resolving deltas: 100% (16120/16120), done.
/content/mmaction2
Obtaining file:///content/mmaction2
  Prepa

In [1]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# Check MMEngine installation
from mmengine.utils.dl_utils import collect_env
print(collect_env())

2.2.1 True
1.2.0
12.3
MSVC 193933521
OrderedDict([('sys.platform', 'win32'), ('Python', '3.8.18 (default, Sep 11 2023, 13:39:12) [MSC v.1916 64 bit (AMD64)]'), ('CUDA available', True), ('MUSA available', False), ('numpy_random_seed', 2147483648), ('GPU 0', 'NVIDIA GeForce RTX 2060'), ('CUDA_HOME', 'C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v12.3'), ('NVCC', 'Cuda compilation tools, release 12.3, V12.3.107'), ('MSVC', 'n/a, reason: fileno'), ('PyTorch', '2.2.1'), ('PyTorch compiling details', 'PyTorch built with:\n  - C++ Version: 201703\n  - MSVC 192930151\n  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v3.3.2 (Git Hash 2dc95a2ad0841e29db8b22fbccaf3e5da7992b01)\n  - OpenMP 2019\n  - LAPACK is enabled (usually provided by MKL)\n  - CPU capability usage: AVX2\n  - CUDA Runtime 12.1\n  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-ge

## Perform inference with a MMAction2 recognizer
MMAction2 already provides high level APIs to do inference and training.

In [3]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth \
      -O checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth

'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [6]:
# MMAction2와 MMEngine 라이브러리에서 필요한 함수와 클래스를 임포트합니다.
from mmaction.apis import inference_recognizer, init_recognizer
from mmengine import Config

# 모델 구성 파일을 선택합니다. 이 구성 파일은 모델의 구조와 학습 설정을 정의합니다.
# 여기서는 ImageNet으로 사전 훈련된 ResNet-50을 백본으로 사용하는 TSN 모델의 구성 파일입니다.
config = 'configs/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py'

# 선택한 구성 파일을 로드하여 Config 객체를 생성합니다.
config = Config.fromfile(config)

# 학습된 모델의 체크포인트 파일 경로를 지정합니다. 
# 이 체크포인트는 모델이 Kinetics-400 데이터셋에서 학습된 가중치를 포함합니다.
checkpoint = 'checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

# init_recognizer 함수를 사용하여 모델을 초기화합니다. 이때, 구성 파일, 체크포인트 파일, 
# 그리고 모델을 실행할 디바이스('cuda:0'은 첫 번째 GPU를 의미합니다)를 지정합니다.
model = init_recognizer(config, checkpoint, device='cuda:0')


Loads checkpoint by local backend from path: checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth


In [7]:
# 필요한 함수를 임포트합니다.
from operator import itemgetter

# 인식을 수행할 비디오 파일의 경로를 지정합니다.
video = 'demo/demo.mp4'
# Kinetics-400 데이터셋의 라벨 매핑 파일 경로를 지정합니다.
label = 'tools/data/kinetics/label_map_k400.txt'

# 초기화한 모델을 사용하여 비디오에 대한 인식 추론을 실행합니다.
results = inference_recognizer(model, video)

# 결과에서 예측 점수를 추출하고, 리스트로 변환합니다.
pred_scores = results.pred_score.tolist()
# 예측 점수와 해당 인덱스를 튜플로 묶어줍니다.
score_tuples = tuple(zip(range(len(pred_scores)), pred_scores))
# 예측 점수를 기준으로 튜플을 내림차순으로 정렬합니다.
score_sorted = sorted(score_tuples, key=itemgetter(1), reverse=True)
# 정렬된 리스트에서 상위 5개의 라벨 인덱스와 점수를 선택합니다.
top5_label = score_sorted[:5]

# 라벨 파일을 열어 각 라인을 읽고, 줄바꿈 문자를 제거합니다.
labels = open(label).readlines()
labels = [x.strip() for x in labels]
# 상위 5개의 예측 결과에 대응하는 라벨 이름과 점수를 튜플로 묶어 리스트로 만듭니다.
results = [(labels[k[0]], k[1]) for k in top5_label]


03/14 17:59:24 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
03/14 17:59:24 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.


In [8]:
# 상위 5개 액션 라벨과 그 점수를 출력하는 문장입니다.
print('The top-5 labels with corresponding scores are:')

# results 리스트를 반복하여 각 결과를 출력합니다.
for result in results:
    # result[0]은 액션 라벨의 이름을, result[1]은 해당 액션의 예측 점수를 나타냅니다.
    # 예: 'running': 0.95
    print(f'{result[0]}: ', result[1])


The top-5 labels with corresponding scores are:
arm wrestling:  1.0
rock scissors paper:  6.434453414527752e-09
shaking hands:  2.7599860175087088e-09
clapping:  1.345463851443185e-09
massaging feet:  5.555100823784187e-10


이 코드 및 명령어 시퀀스는 MMAction2를 사용하여 사용자 지정 데이터셋에 대한 비디오 액션 인식 모델을 훈련하는 과정을 설명합니다.   
과정은 크게 비디오 데이터의 준비, 모델의 구성 수정, 그리고 새로운 인식기의 학습으로 나눌 수 있습니다. 아래는 각 단계별 상세 설명입니다:

### 비디오 데이터 준비
- **비디오 파일 경로 지정**: `video = 'demo/demo.mp4'` 명령어는 모델이 인식을 수행할 비디오 파일의 위치를 지정합니다.
- **라벨 매핑 파일 경로 지정**: `label = 'tools/data/kinetics/label_map_k400.txt'` 명령어는 Kinetics-400 데이터셋의 라벨을 매핑하는 파일의 위치를 지정합니다. 이 파일은 비디오에서 인식된 액션들의 이름을 포함하고 있습니다.

### 모델을 이용한 비디오 인식
- **모델을 사용하여 비디오에서 액션 인식 수행**: `results = inference_recognizer(model, video)`는 초기화된 모델을 사용하여 지정된 비디오 파일에서 액션 인식을 수행하고 결과를 반환합니다.
- **예측 점수 추출 및 정렬**: 추출된 예측 점수를 리스트로 변환하고, 이를 내림차순으로 정렬하여 상위 5개의 결과를 선택합니다. 이 과정은 인식된 액션들 중에서 가장 확률이 높은 상위 5개를 식별합니다.

### 훈련 데이터셋 다운로드 및 준비
- **Kinetics-400 Tiny 데이터셋 다운로드**: 이 부분의 코드는 Kinetics-400 데이터셋의 작은 버전을 다운로드하고 압축을 해제합니다. 이 작은 데이터셋은 30개의 훈련 비디오와 10개의 테스트 비디오를 포함하고 있으며, 새로운 모델을 훈련하는 데 사용됩니다.
- **데이터셋 구조 확인**: `!tree kinetics400_tiny` 명령어는 다운로드한 데이터셋의 디렉토리 구조를 확인하는 데 사용됩니다.
- **데이터셋 주석 포맷 확인**: `!cat kinetics400_tiny/kinetics_tiny_train_video.txt` 명령어는 훈련 데이터셋의 주석 포맷을 확인하는 데 사용됩니다. 이는 모델이 학습을 수행할 때 어떤 비디오가 어떤 라벨을 가지는지 이해하는 데 필요합니다.

### 사용자 지정 데이터셋에 대한 모델 훈련
이 과정은 사용자 지정 데이터셋을 지원하기 위한 준비, 구성 수정, 그리고 새로운 인식기 훈련으로 구성됩니다. 여기서는 Kinetics-400 Tiny 데이터셋을 사용하는 예시를 통해 새 데이터셋을 다루는 방법과 모델을 훈련시키는 과정을 설명합니다.

이 과정은 MMAction2를 사용하여 비디오 액션 인식 모델을 학습시키고자 할 때 필요한 기본 단계를 다룹니다. 데이터 준비부터 모델 학습까지, 각 단계는 데이터셋의 특성과 프로젝트의 요구 사항에 맞춰 조정될 수 있습니다.

## Train a recognizer on customized dataset

To train a new recognizer, there are usually three things to do:
1. Support a new dataset
2. Modify the config
3. Train a new recognizer

### Support a new dataset

In this tutorial, we gives an example to convert the data into the format of existing datasets. Other methods and more advanced usages can be found in the [doc](/docs/tutorials/new_dataset.md)

Firstly, let's download a tiny dataset obtained from [Kinetics-400](https://deepmind.com/research/open-source/open-source-datasets/kinetics/). We select 30 videos with their labels as train dataset and 10 videos with their labels as test dataset.

이 코드는 사용자 지정 데이터셋으로 새로운 액션 인식 모델을 훈련시키는 과정을 설명하는 예시입니다. Kinetics-400 데이터셋의 축소된 버전을 다운로드하여 사용하여, 모델 훈련을 위한 데이터 준비 과정을 보여줍니다. 아래는 각 단계별 설명입니다:

1. **새 데이터셋 지원**: 이 단계에서는 새 데이터셋을 MMAction2 또는 관련 머신러닝 프레임워크가 인식할 수 있는 형식으로 변환하는 작업을 합니다. 예시에서는 Kinetics-400의 축소 버전을 사용하지만, 실제 프로젝트에서는 자체 데이터셋을 이 형식으로 준비해야 할 수 있습니다.

2. **구성 수정**: 모델의 구성 파일(config)을 수정하여 새 데이터셋에 맞게 조정합니다. 이는 데이터 경로, 모델 파라미터, 학습 루틴 등을 포함할 수 있습니다.

3. **새 인식기 훈련**: 준비된 데이터셋과 수정된 구성을 바탕으로 모델 학습을 시작합니다.

### 데이터셋 다운로드 및 준비 과정



- **데이터셋 다운로드 및 압축 해제**: 주어진 URL에서 Kinetics-400의 축소 버전 데이터셋을 다운로드하고 압축을 해제합니다. 이 데이터셋에는 훈련용으로 30개의 비디오와 테스트용으로 10개의 비디오가 포함되어 있습니다.

  


In [7]:
# # download, decompress the data
# !rm kinetics400_tiny.zip* # 기존에 다운로드된 파일이 있을 경우 삭제
# !rm -rf kinetics400_tiny # 기존에 압축 해제된 디렉토리가 있을 경우 삭제
# !wget https://download.openmmlab.com/mmaction/kinetics400_tiny.zip # 데이터셋 다운로드
# !unzip kinetics400_tiny.zip > /dev/null # 압축 해제, 출력은 생략


import requests, zipfile, io, os

# 기존에 다운로드된 파일 삭제
if os.path.exists('kinetics400_tiny.zip'):
    os.remove('kinetics400_tiny.zip')

# 기존에 압축 해제된 디렉토리 삭제
if os.path.exists('kinetics400_tiny'):
    os.rmdir('kinetics400_tiny')

# 데이터셋 다운로드
url = 'https://download.openmmlab.com/mmaction/kinetics400_tiny.zip'
response = requests.get(url)
with open("kinetics400_tiny.zip", "wb") as f:
    f.write(response.content)

# 압축 해제
with zipfile.ZipFile("kinetics400_tiny.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

print("Download and extraction complete.")


Download and extraction complete.


- **디렉토리 구조 확인**: `tree` 명령어를 사용하여 다운로드한 데이터셋의 디렉토리 구조를 확인합니다. 이는 훈련 및 테스트 데이터가 올바르게 구성되었는지 확인하는 데 도움이 됩니다.


In [8]:
# # Check the directory structure of the tiny data

# # Install tree first
# !apt-get -q install tree # tree 명령어 설치
# !tree kinetics400_tiny # 데이터셋 디렉토리 구조 출력


import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print(f'{subindent}{f}')

# 경로를 원하는 데이터셋 디렉토리로 변경하세요.
list_files('kinetics400_tiny')


kinetics400_tiny/
    kinetics_tiny_train_video.txt
    kinetics_tiny_val_video.txt
    train/
        27_CSXByd3s.mp4
        34XczvTaRiI.mp4
        A-wiliK50Zw.mp4
        D32_1gwq35E.mp4
        D92m0HsHjcQ.mp4
        DbX8mPslRXg.mp4
        FMlSTTpN3VY.mp4
        h10B9SVE-nk.mp4
        h2YqqUhnR34.mp4
        iRuyZSKhHRg.mp4
        IyfILH9lBRo.mp4
        kFC3KY2bOP8.mp4
        LvcFDgCAXQs.mp4
        O46YA8tI530.mp4
        oMrZaozOvdQ.mp4
        oXy-e_P_cAI.mp4
        P5M-hAts7MQ.mp4
        phDqGd0NKoo.mp4
        PnOe3GZRVX8.mp4
        R8HXQkdgKWA.mp4
        RqnKtCEoEcA.mp4
        soEcZZsBmDs.mp4
        TkkZPZHbAKA.mp4
        T_TMNGzVrDk.mp4
        WaS0qwP46Us.mp4
        Wh_YPQdH1Zg.mp4
        WWP5HZJsg-o.mp4
        xGY2dP0YUjA.mp4
        yLC9CtWU5ws.mp4
        ZQV4U2KQ370.mp4
    val/
        0pVGiAU6XEA.mp4
        AQrbRSnRt8M.mp4
        b6Q_b7vgc7Q.mp4
        ddvJ6-faICE.mp4
        IcLztCtvhb8.mp4
        ik4BW3-SCts.mp4
        jqRrH30V0k4.mp4
        

In [10]:
# # After downloading the data, we need to check the annotation format
# !cat kinetics400_tiny/kinetics_tiny_train_video.txt # 주석 파일 내용 출력


# 파일 경로를 정확하게 지정해주세요.
file_path = 'kinetics400_tiny/kinetics_tiny_train_video.txt'

with open(file_path, 'r') as file:
    content = file.read()
    print(content)


D32_1gwq35E.mp4 0
iRuyZSKhHRg.mp4 1
oXy-e_P_cAI.mp4 0
34XczvTaRiI.mp4 1
h2YqqUhnR34.mp4 0
O46YA8tI530.mp4 0
kFC3KY2bOP8.mp4 1
WWP5HZJsg-o.mp4 1
phDqGd0NKoo.mp4 1
yLC9CtWU5ws.mp4 0
27_CSXByd3s.mp4 1
IyfILH9lBRo.mp4 1
T_TMNGzVrDk.mp4 1
TkkZPZHbAKA.mp4 0
PnOe3GZRVX8.mp4 1
soEcZZsBmDs.mp4 1
FMlSTTpN3VY.mp4 1
WaS0qwP46Us.mp4 0
A-wiliK50Zw.mp4 1
oMrZaozOvdQ.mp4 1
ZQV4U2KQ370.mp4 0
DbX8mPslRXg.mp4 1
h10B9SVE-nk.mp4 1
P5M-hAts7MQ.mp4 0
R8HXQkdgKWA.mp4 0
D92m0HsHjcQ.mp4 0
RqnKtCEoEcA.mp4 0
LvcFDgCAXQs.mp4 0
xGY2dP0YUjA.mp4 0
Wh_YPQdH1Zg.mp4 0



According to the format defined in [`VideoDataset`](./datasets/video_dataset.py), each line indicates a sample video with the filepath and label, which are split with a whitespace.

### Modify the config

In the next step, we need to modify the config for the training.
To accelerate the process, we finetune a recognizer using a pre-trained recognizer.

In [12]:
# MMAction2 설정 파일을 로드합니다. 이 설정 파일은 TSN(Temporal Segment Networks) 모델이 
# ImageNet으로 사전 훈련된 ResNet-50을 기반으로 하며, Kinetics-400 데이터셋에 대해 
# 훈련될 때 사용되는 구성을 정의합니다.
# 여기서:
# - 'tsn_imagenet-pretrained-r50'는 ImageNet으로 사전 훈련된 ResNet-50을 사용한다는 것을 의미합니다.
# - '8xb32'는 8개의 GPUs를 사용하며, 각 GPU 당 배치 크기가 32임을 나타냅니다.
# - '1x1x3-100e'는 학습 스케쥴을 나타내며, 보통 이는 학습률 변경 지점과 총 에폭 수(여기서는 100 에폭)를 의미합니다.
# - 'kinetics400-rgb'는 Kinetics-400 데이터셋의 RGB 비디오 프레임을 사용한다는 것을 의미합니다.
cfg = Config.fromfile('./configs/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py')


Given a config that trains a TSN model on kinetics400-full dataset, we need to modify some values to use it for training TSN on Kinetics400-tiny dataset.


In [13]:
from mmengine.runner import set_random_seed

# 학습 및 검증 데이터셋의 루트 경로를 설정합니다.
cfg.data_root = 'kinetics400_tiny/train/'
cfg.data_root_val = 'kinetics400_tiny/val/'

# 학습 및 검증 데이터셋의 어노테이션 파일 경로를 설정합니다.
cfg.ann_file_train = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.ann_file_val = 'kinetics400_tiny/kinetics_tiny_val_video.txt'

# 테스트 데이터셋의 어노테이션 파일 및 비디오 파일 경로를 설정합니다.
cfg.test_dataloader.dataset.ann_file = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.test_dataloader.dataset.data_prefix.video = 'kinetics400_tiny/val/'

# 학습 데이터셋의 어노테이션 파일 및 비디오 파일 경로를 설정합니다.
cfg.train_dataloader.dataset.ann_file = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.train_dataloader.dataset.data_prefix.video = 'kinetics400_tiny/train/'

# 검증 데이터셋의 어노테이션 파일 및 비디오 파일 경로를 설정합니다.
cfg.val_dataloader.dataset.ann_file = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.val_dataloader.dataset.data_prefix.video  = 'kinetics400_tiny/val/'

# 모델의 분류기 머리 부분에서 클래스의 수를 수정합니다. 여기서는 2로 설정합니다.
cfg.model.cls_head.num_classes = 2

# 사전 훈련된 TSN 모델을 사용합니다.
cfg.load_from = './checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

# 파일과 로그를 저장할 작업 디렉토리를 설정합니다.
cfg.work_dir = './tutorial_exps'

# 원래 학습률(LR)은 8-GPU 학습에 맞춰 설정되어 있습니다.
# 하나의 GPU만 사용하기 때문에 이를 8로 나눕니다.
cfg.train_dataloader.batch_size = cfg.train_dataloader.batch_size // 16
cfg.val_dataloader.batch_size = cfg.val_dataloader.batch_size // 16
cfg.optim_wrapper.optimizer.lr = cfg.optim_wrapper.optimizer.lr / 8 / 16

# 최대 에폭 수를 10으로 설정합니다.
cfg.train_cfg.max_epochs = 10

# 데이터 로더의 작업자 수를 설정합니다.
cfg.train_dataloader.num_workers = 2
cfg.val_dataloader.num_workers = 2
cfg.test_dataloader.num_workers = 2

# 훈련을 위해 최종 사용된 설정을 로거로 초기화하고 확인합니다.
print(f'Config:\n{cfg.pretty_text}')


Config:
ann_file_train = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
ann_file_val = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
auto_scale_lr = dict(base_batch_size=256, enable=False)
data_root = 'kinetics400_tiny/train/'
data_root_val = 'kinetics400_tiny/val/'
dataset_type = 'VideoDataset'
default_hooks = dict(
    checkpoint=dict(
        interval=3, max_keep_ckpts=3, save_best='auto', type='CheckpointHook'),
    logger=dict(ignore_last=False, interval=20, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    runtime_info=dict(type='RuntimeInfoHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    sync_buffers=dict(type='SyncBuffersHook'),
    timer=dict(type='IterTimerHook'))
default_scope = 'mmaction'
env_cfg = dict(
    cudnn_benchmark=False,
    dist_cfg=dict(backend='nccl'),
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0))
file_client_args = dict(io_backend='disk')
load_from = './checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_

### Train a new recognizer

Finally, lets initialize the dataset and recognizer, then train a new recognizer!

In [14]:
import os.path as osp
import mmengine
from mmengine.runner import Runner

# 작업 디렉토리를 생성합니다. 모델 학습 중 생성되는 파일과 로그를 저장하기 위한 디렉토리입니다.
mmengine.mkdir_or_exist(osp.abspath(cfg.work_dir))

# 설정 파일을 기반으로 학습 실행자(Runner)를 구축합니다. Runner는 학습 과정을 관리합니다.
runner = Runner.from_cfg(cfg)

# 학습을 시작합니다. 이 과정은 데이터셋을 사용하여 모델을 학습하고, 설정된 에폭 수만큼 반복합니다.
runner.train()


03/14 14:49:07 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.8.18 (default, Sep 11 2023, 13:39:12) [MSC v.1916 64 bit (AMD64)]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1383582061
    GPU 0: NVIDIA GeForce RTX 4070 Laptop GPU
    CUDA_HOME: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.3
    NVCC: Cuda compilation tools, release 12.3, V12.3.103
    MSVC: n/a, reason: fileno
    PyTorch: 2.2.1
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 201703
  - MSVC 192930151
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.2 (Git Hash 2dc95a2ad0841e29db8b22fbccaf3e5da7992b01)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_5

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\chanj/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth


03/14 14:49:18 - mmengine - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.weight', 'fc.bias'}
Loads checkpoint by local backend from path: ./checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
The model and loaded state dict do not match exactly

size mismatch for cls_head.fc_cls.weight: copying a param with shape torch.Size([400, 2048]) from checkpoint, the shape in current model is torch.Size([2, 2048]).
size mismatch for cls_head.fc_cls.bias: copying a param with shape torch.Size([400]) from checkpoint, the shape in current model is torch.Size([2]).
03/14 14:49:18 - mmengine - INFO - Load checkpoint from ./checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
03/14 14:49:18 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
03/14 14:49:18 - mmengine - INFO - Checkpoints will be saved to c:\Users\chanj\workspace\Anaconda f

Recognizer2D(
  (data_preprocessor): ActionDataPreprocessor()
  (backbone): ResNet(
    (conv1): ConvModule(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): ConvModule(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activate): ReLU(inplace=True)
        )
        (conv2): ConvModule(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activate): ReLU(inplace=True)
        

### Understand the log
From the log, we can have a basic understanding the training process and know how well the recognizer is trained.

Firstly, the ResNet-50 backbone pre-trained on ImageNet is loaded, this is a common practice since training from scratch is more cost. The log shows that all the weights of the ResNet-50 backbone are loaded except the `fc.bias` and `fc.weight`.

Second, since the dataset we are using is small, we loaded a TSN model and finetune it for action recognition.
The original TSN is trained on original Kinetics-400 dataset which contains 400 classes but Kinetics-400 Tiny dataset only have 2 classes. Therefore, the last FC layer of the pre-trained TSN for classification has different weight shape and is not used.

Third, after training, the recognizer is evaluated by the default evaluation. The results show that the recognizer achieves 100% top1 accuracy and 100% top5 accuracy on the val dataset,

Not bad!

### 로그 이해하기
학습 과정 중 생성된 로그는 모델 학습의 진행 상황과 성능을 이해하는 데 도움을 줍니다.

- **첫째**, ImageNet으로 사전 훈련된 ResNet-50 백본이 로드됩니다. 이는 처음부터 훈련하는 것보다 비용이 적게 드는 일반적인 관행입니다. 로그에서는 ResNet-50 백본의 모든 가중치가 로드되었음을 보여주지만, `fc.bias`와 `fc.weight`는 제외됩니다.

- **둘째**, 사용 중인 데이터셋이 작기 때문에, TSN 모델을 로드하고 액션 인식을 위해 미세 조정(finetune)합니다. 원래의 TSN은 400개 클래스를 포함하는 원본 Kinetics-400 데이터셋에서 훈련되었지만, Kinetics-400 Tiny 데이터셋은 오직 2개의 클래스만을 가지고 있습니다. 따라서, 사전 훈련된 TSN의 분류를 위한 마지막 FC 계층은 가중치 형태가 다르므로 사용되지 않습니다.

- **셋째**, 훈련 후, 모델은 기본 평가로 평가됩니다. 결과는 평가 데이터셋에서 100%의 top1 정확도와 100%의 top5 정확도를 달성했다고 보여줍니다.

## Test the trained recognizer

After finetuning the recognizer, let's check the prediction results!

### 훈련된 인식기 테스트하기
모델을 미세 조정한 후, 예측 결과를 확인해 보겠습니다.

In [15]:
# 훈련된 모델을 사용하여 테스트 데이터셋에 대한 예측을 수행합니다.
runner.test()


03/14 14:54:06 - mmengine - INFO - Epoch(test) [10/10]    acc/top1: 0.9000  acc/top5: 1.0000  acc/mean1: 0.9000  data_time: 0.4100  time: 1.1401


{'acc/top1': 0.9, 'acc/top5': 1.0, 'acc/mean1': 0.9}

이 단계는 모델이 얼마나 잘 학습되었는지, 즉 새로운 데이터에 대한 모델의 일반화 능력을 평가하는 과정입니다.